In [34]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import ast
from sklearn.preprocessing import MultiLabelBinarizer
from bs4 import BeautifulSoup
import re
import string

In [26]:
pip install beautifulsoup4

You should consider upgrading via the '/opt/anaconda3/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
df = pd.read_csv('/Users/youziya/Library/Group Containers/UBF8T346G9.OneDriveStandaloneSuite/OneDrive - York University.noindex/OneDrive - York University/MBAN 6090 Analytics Project/jobs_info.csv')

In [3]:
df.head()

,Unnamed: 0,organization_name,title,locations,organization_topics,organization_industry_tags,url,has_description,description
0,1,Vidyard,"Director of Sales, Enterprise Acquisition","['United States', 'Canada', 'Remote']","['B2B', 'Media & entertainment', 'Productivity...","['Media', 'Software']",https://www1.communitech.ca/companies/vidyard/...,True,"<div data-testid=""careerPage""><p>At Vidyard, w..."
1,2,Hootsuite,Senior Software Developer | Développeur logici...,"['Toronto, ON, Canada', 'Vancouver, BC, Canada']","['500+', 'Advertising/marketing', 'B2B', 'Soft...","['Information Technology', 'Media', 'Sales and...",https://www1.communitech.ca/companies/hootsuit...,True,"<div data-testid=""careerPage""><p><span><em><st..."
2,3,Semios,Chemistry Co-op,"['Vancouver, BC, Canada']","['Agriculture', 'B2B', 'C100 alumni', 'Data & ...",['Agriculture and Farming'],https://www1.communitech.ca/companies/semios/j...,True,"<div data-testid=""careerPage""><p><strong>Who w..."
3,4,Axonify,Senior Talent Acquisition Partner (15 months c...,"['Toronto, ON, Canada', 'Waterloo, ON, Canada']","['B2B', 'Data & analytics', 'Education', 'Fema...","['Education', 'Information Technology', 'Softw...",https://www1.communitech.ca/companies/axonify/...,True,"<div data-testid=""careerPage""><section><div>Ax..."
4,5,Deep Genomics,(Senior) Research Scientist - Statistical Gene...,"['Toronto, ON, Canada']","['50-99', 'AI/ML', 'B2B', 'Biotechnology', 'Me...","['Biotechnology', 'DeepTech', 'Health', 'Softw...",https://www1.communitech.ca/companies/deep-gen...,True,"<div data-testid=""careerPage""><section><div><b..."


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4182 entries, 0 to 4181
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   Unnamed: 0                  4182 non-null   int64 
 1   organization_name           4182 non-null   object
 2   title                       4182 non-null   object
 3   locations                   4182 non-null   object
 4   organization_topics         4182 non-null   object
 5   organization_industry_tags  4182 non-null   object
 6   url                         4182 non-null   object
 7   has_description             4182 non-null   bool  
 8   description                 4159 non-null   object
dtypes: bool(1), int64(1), object(7)
memory usage: 265.6+ KB


In [5]:
df = df.drop(columns=['Unnamed: 0'])


In [6]:
import pandas as pd

# Assuming df is your DataFrame

# Define a parsing function for list-based locations
def parse_location_list(location_list):
    # Flatten the list into a single string separated by commas
    location_str = ', '.join(location_list)
    
    # Now you can use the same parsing logic as before
    city, state_province, country, is_remote = '', '', '', False

    # Check for 'Remote'
    if 'Remote' in location_str:
        is_remote = True

    # Split the location string
    parts = location_str.replace('Remote', '').split(', ')
    parts = [part for part in parts if part]  # Remove empty parts

    # Update city, state_province, and country based on the number of parts
    if len(parts) >= 3:
        # Assuming the last part is always the country
        country = parts[-1]
        state_province = parts[-2]
        city = ', '.join(parts[:-2])  # Combine the rest as the city
    elif len(parts) == 2:
        state_province, country = parts
    elif len(parts) == 1:
        country = parts[0]

    return city, state_province, country, is_remote

# Apply the parsing function to each list in the 'locations' column
df[['City', 'State/Province', 'Country', 'Is_Remote']] = df['locations'].apply(lambda x: parse_location_list(x)).tolist()

# Display the result
print(df.head())


  organization_name                                              title  \
0           Vidyard          Director of Sales, Enterprise Acquisition   
1         Hootsuite  Senior Software Developer | Développeur logici...   
2            Semios                                    Chemistry Co-op   
3           Axonify  Senior Talent Acquisition Partner (15 months c...   
4     Deep Genomics  (Senior) Research Scientist - Statistical Gene...   

                                          locations  \
0             ['United States', 'Canada', 'Remote']   
1  ['Toronto, ON, Canada', 'Vancouver, BC, Canada']   
2                         ['Vancouver, BC, Canada']   
3   ['Toronto, ON, Canada', 'Waterloo, ON, Canada']   
4                           ['Toronto, ON, Canada']   

                                 organization_topics  \
0  ['B2B', 'Media & entertainment', 'Productivity...   
1  ['500+', 'Advertising/marketing', 'B2B', 'Soft...   
2  ['Agriculture', 'B2B', 'C100 alumni', 'Data & ...   


In [12]:
df.head()

,organization_name,title,locations,organization_topics,organization_industry_tags,url,has_description,description,City,State/Province,Country,Is_Remote,consolidatedJobTitle
0,Vidyard,"Director of Sales, Enterprise Acquisition","['United States', 'Canada', 'Remote']","['B2B', 'Media & entertainment', 'Productivity...","['Media', 'Software']",https://www1.communitech.ca/companies/vidyard/...,True,"<div data-testid=""careerPage""><p>At Vidyard, w...","[, ', U, n, i, t, e, d, , S, t, a, t, e, s, '...",',],False,Director
1,Hootsuite,Senior Software Developer | Développeur logici...,"['Toronto, ON, Canada', 'Vancouver, BC, Canada']","['500+', 'Advertising/marketing', 'B2B', 'Soft...","['Information Technology', 'Media', 'Sales and...",https://www1.communitech.ca/companies/hootsuit...,True,"<div data-testid=""careerPage""><p><span><em><st...","[, ', T, o, r, o, n, t, o, ,, , O, N, ,, , C...",',],False,Senior
2,Semios,Chemistry Co-op,"['Vancouver, BC, Canada']","['Agriculture', 'B2B', 'C100 alumni', 'Data & ...",['Agriculture and Farming'],https://www1.communitech.ca/companies/semios/j...,True,"<div data-testid=""careerPage""><p><strong>Who w...","[, ', V, a, n, c, o, u, v, e, r, ,, , B, C, ,...",',],False,Co-op
3,Axonify,Senior Talent Acquisition Partner (15 months c...,"['Toronto, ON, Canada', 'Waterloo, ON, Canada']","['B2B', 'Data & analytics', 'Education', 'Fema...","['Education', 'Information Technology', 'Softw...",https://www1.communitech.ca/companies/axonify/...,True,"<div data-testid=""careerPage""><section><div>Ax...","[, ', T, o, r, o, n, t, o, ,, , O, N, ,, , C...",',],False,Senior
4,Deep Genomics,(Senior) Research Scientist - Statistical Gene...,"['Toronto, ON, Canada']","['50-99', 'AI/ML', 'B2B', 'Biotechnology', 'Me...","['Biotechnology', 'DeepTech', 'Health', 'Softw...",https://www1.communitech.ca/companies/deep-gen...,True,"<div data-testid=""careerPage""><section><div><b...","[, ', T, o, r, o, n, t, o, ,, , O, N, ,, , C...",',],False,Senior


In [23]:
df['organization_industry_tags']

0                                   ['Media', 'Software']
1       ['Information Technology', 'Media', 'Sales and...
2                             ['Agriculture and Farming']
3       ['Education', 'Information Technology', 'Softw...
4       ['Biotechnology', 'DeepTech', 'Health', 'Softw...
                              ...                        
4177    ['Ecommerce & Marketplace', 'Finance', 'Softwa...
4178      ['Hardware', 'Sales and Marketing', 'Software']
4179                  ['Sales and Marketing', 'Software']
4180               ['Information Technology', 'Software']
4181               ['Information Technology', 'Software']
Name: organization_industry_tags, Length: 4182, dtype: object

In [17]:
def reformat_location(location_list):
    # Join the characters in the list to form a string
    return ''.join(location_list).strip("[]' ")

# Apply the reformatting function to the 'locations' column
df['locations'] = df['locations'].apply(reformat_location)

# Now you can apply your original parsing function
df[['City', 'State/Province', 'Country', 'Is_Remote']] = df['locations'].apply(lambda x: parse_location_list(x.split(', '))).tolist()

# Display the result
df[['locations', 'City', 'State/Province', 'Country', 'Is_Remote']].head()


,locations,City,State/Province,Country,Is_Remote
0,"United States', 'Canada', 'Remote",United States','Canada',',True
1,"Toronto, ON, Canada', 'Vancouver, BC, Canada","Toronto, ON, Canada', 'Vancouver",BC,Canada,False
2,"Vancouver, BC, Canada",Vancouver,BC,Canada,False
3,"Toronto, ON, Canada', 'Waterloo, ON, Canada","Toronto, ON, Canada', 'Waterloo",ON,Canada,False
4,"Toronto, ON, Canada",Toronto,ON,Canada,False


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4182 entries, 0 to 4181
Data columns (total 13 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   organization_name           4182 non-null   object
 1   title                       4182 non-null   object
 2   locations                   4182 non-null   object
 3   organization_topics         4182 non-null   object
 4   organization_industry_tags  4182 non-null   object
 5   url                         4182 non-null   object
 6   has_description             4182 non-null   bool  
 7   description                 4159 non-null   object
 8   City                        4182 non-null   object
 9   State/Province              4182 non-null   object
 10  Country                     4182 non-null   object
 11  Is_Remote                   4182 non-null   object
 12  consolidatedJobTitle        4182 non-null   object
dtypes: bool(1), object(12)
memory usage: 396.3+ KB


In [19]:
df['City']

0                         United States'
1       Toronto, ON, Canada', 'Vancouver
2                              Vancouver
3        Toronto, ON, Canada', 'Waterloo
4                                Toronto
                      ...               
4177                             Toronto
4178                             Toronto
4179                                    
4180                                    
4181                                    
Name: City, Length: 4182, dtype: object

In [29]:
df.dropna()

,organization_name,title,locations,organization_topics,organization_industry_tags,url,has_description,description,City,State/Province,Country,Is_Remote,consolidatedJobTitle
0,Vidyard,"Director of Sales, Enterprise Acquisition","United States', 'Canada', 'Remote","['B2B', 'Media & entertainment', 'Productivity...","[Media, Software]",https://www1.communitech.ca/companies/vidyard/...,True,"<div data-testid=""careerPage""><p>At Vidyard, w...",United States','Canada',',True,Director
1,Hootsuite,Senior Software Developer | Développeur logici...,"Toronto, ON, Canada', 'Vancouver, BC, Canada","['500+', 'Advertising/marketing', 'B2B', 'Soft...","[Information Technology, Media, Sales and Mark...",https://www1.communitech.ca/companies/hootsuit...,True,"<div data-testid=""careerPage""><p><span><em><st...","Toronto, ON, Canada', 'Vancouver",BC,Canada,False,Senior
2,Semios,Chemistry Co-op,"Vancouver, BC, Canada","['Agriculture', 'B2B', 'C100 alumni', 'Data & ...",[Agriculture and Farming],https://www1.communitech.ca/companies/semios/j...,True,"<div data-testid=""careerPage""><p><strong>Who w...",Vancouver,BC,Canada,False,Co-op
3,Axonify,Senior Talent Acquisition Partner (15 months c...,"Toronto, ON, Canada', 'Waterloo, ON, Canada","['B2B', 'Data & analytics', 'Education', 'Fema...","[Education, Information Technology, Software]",https://www1.communitech.ca/companies/axonify/...,True,"<div data-testid=""careerPage""><section><div>Ax...","Toronto, ON, Canada', 'Waterloo",ON,Canada,False,Senior
4,Deep Genomics,(Senior) Research Scientist - Statistical Gene...,"Toronto, ON, Canada","['50-99', 'AI/ML', 'B2B', 'Biotechnology', 'Me...","[Biotechnology, DeepTech, Health, Software]",https://www1.communitech.ca/companies/deep-gen...,True,"<div data-testid=""careerPage""><section><div><b...",Toronto,ON,Canada,False,Senior
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4177,Payment Source,hr@paymentsource.ca,"Toronto, ON, Canada","['B2B', 'Consumer & lifestyle', 'Finance', 'Sa...","[Ecommerce & Marketplace, Finance, Software]",https://www1.communitech.ca/companies/payment-...,True,"<div data-testid=""careerPage""><h2 class=""uk-he...",Toronto,ON,Canada,False,Other
4178,Vantage,Intermediate/Senior Full Stack Developer,"Toronto, ON, Canada","['Advertising/marketing', 'B2B', 'C100 alumni'...","[Hardware, Sales and Marketing, Software]",https://www1.communitech.ca/companies/vantage/...,True,"<div data-testid=""careerPage""><div class=""cont...",Toronto,ON,Canada,False,Senior
4179,StackAdapt,Data Scientist / Senior Data Scientist,Canada,"['Advertising/marketing', 'B2B', 'C100 alumni'...","[Sales and Marketing, Software]",https://www1.communitech.ca/companies/stackada...,True,"<div data-testid=""careerPage""><section><div>St...",,,Canada,False,Senior
4180,Replicon,Product Support Associate,"India', 'Remote","['500+', 'AI/ML', 'B2B', 'Productivity', 'SaaS...","[Information Technology, Software]",https://www1.communitech.ca/companies/replicon...,True,"<div data-testid=""careerPage""><div class=""cont...",,India',',True,Other


In [11]:
def consolidate_job_titles(job_title):
    # Define broad categories with keywords
    categories = {
        'Director': ['Director'],
        'Senior': ['Senior'],
        'Co-op': ['Co-op']
    }

    # Default category
    consolidated_category = 'Other'

    # Check for each keyword in the job title
    for category, keywords in categories.items():
        if any(keyword in job_title for keyword in keywords):
            consolidated_category = category
            break

    return consolidated_category

# Apply the function to the 'title' column
df['consolidatedJobTitle'] = df['title'].apply(consolidate_job_titles)

# Display the first few rows to verify the changes
df[['title', 'consolidatedJobTitle']].head()


,title,consolidatedJobTitle
0,"Director of Sales, Enterprise Acquisition",Director
1,Senior Software Developer | Développeur logici...,Senior
2,Chemistry Co-op,Co-op
3,Senior Talent Acquisition Partner (15 months c...,Senior
4,(Senior) Research Scientist - Statistical Gene...,Senior


In [ ]:
# Example data (replace this with your actual column)
example_data = [[['Hardware', 'Sales and Marketing', 'Software']]]

# Initializing MultiLabelBinarizer
mlb = MultiLabelBinarizer()

# Transforming the data
transformed_data = mlb.fit_transform(example_data)

# Creating a DataFrame from the transformed data
industry_tags_df = pd.DataFrame(transformed_data, columns=mlb.classes_)

# Displaying the resulting DataFrame
industry_tags_df.head()


In [24]:
#presence 1 and absence 0
# Convert string representation of lists to actual lists
df['organization_industry_tags'] = df['organization_industry_tags'].apply(lambda x: ast.literal_eval(x))

# Applying MultiLabelBinarizer to the 'organization_industry_tags' column
mlb = MultiLabelBinarizer()
transformed_data = mlb.fit_transform(df['organization_industry_tags'])

# Creating a DataFrame from the transformed data
industry_tags_df = pd.DataFrame(transformed_data, columns=mlb.classes_)

# Merging the new DataFrame with the original DataFrame
df_combined = pd.concat([df, industry_tags_df], axis=1)

# Displaying the first few rows of the updated DataFrame
df_combined.head()


,organization_name,title,locations,organization_topics,organization_industry_tags,url,has_description,description,City,State/Province,...,Real Estate,Sales and Marketing,Security,Software,Sports,Sustainability,Telecommunication,Transportation,Travel,Wellness and Beauty
0,Vidyard,"Director of Sales, Enterprise Acquisition","United States', 'Canada', 'Remote","['B2B', 'Media & entertainment', 'Productivity...","[Media, Software]",https://www1.communitech.ca/companies/vidyard/...,True,"<div data-testid=""careerPage""><p>At Vidyard, w...",United States','Canada',...,0,0,0,1,0,0,0,0,0,0
1,Hootsuite,Senior Software Developer | Développeur logici...,"Toronto, ON, Canada', 'Vancouver, BC, Canada","['500+', 'Advertising/marketing', 'B2B', 'Soft...","[Information Technology, Media, Sales and Mark...",https://www1.communitech.ca/companies/hootsuit...,True,"<div data-testid=""careerPage""><p><span><em><st...","Toronto, ON, Canada', 'Vancouver",BC,...,0,1,0,1,0,0,0,0,0,0
2,Semios,Chemistry Co-op,"Vancouver, BC, Canada","['Agriculture', 'B2B', 'C100 alumni', 'Data & ...",[Agriculture and Farming],https://www1.communitech.ca/companies/semios/j...,True,"<div data-testid=""careerPage""><p><strong>Who w...",Vancouver,BC,...,0,0,0,0,0,0,0,0,0,0
3,Axonify,Senior Talent Acquisition Partner (15 months c...,"Toronto, ON, Canada', 'Waterloo, ON, Canada","['B2B', 'Data & analytics', 'Education', 'Fema...","[Education, Information Technology, Software]",https://www1.communitech.ca/companies/axonify/...,True,"<div data-testid=""careerPage""><section><div>Ax...","Toronto, ON, Canada', 'Waterloo",ON,...,0,0,0,1,0,0,0,0,0,0
4,Deep Genomics,(Senior) Research Scientist - Statistical Gene...,"Toronto, ON, Canada","['50-99', 'AI/ML', 'B2B', 'Biotechnology', 'Me...","[Biotechnology, DeepTech, Health, Software]",https://www1.communitech.ca/companies/deep-gen...,True,"<div data-testid=""careerPage""><section><div><b...",Toronto,ON,...,0,0,0,1,0,0,0,0,0,0


In [32]:

def clean_html_description(text):
    # Remove HTML tags using BeautifulSoup
    soup = BeautifulSoup(text, 'html.parser')
    cleaned_text = soup.get_text(separator=' ')

    # Optional: Further cleaning steps can be added here
    # For example, removing extra spaces, converting to lowercase, etc.

    return cleaned_text

# Apply the cleaning function to the description column
df['cleaned_description'] = df['description'].apply(clean_html_description)

# Display the first few rows of the cleaned descriptions
df[['description', 'cleaned_description']].head()


TypeError: object of type 'float' has no len()

In [31]:
def clean_html_description(text):
    # Check if the text is a string
    if not isinstance(text, str):
        return ""  # or any other placeholder you deem appropriate

    # Remove HTML tags using BeautifulSoup
    soup = BeautifulSoup(text, 'html.parser')
    cleaned_text = soup.get_text(separator=' ')

    # Optional: Further cleaning steps can be added here

    return cleaned_text

# Apply the cleaning function to the description column
df['cleaned_description'] = df['description'].apply(clean_html_description)


In [33]:
df[['description', 'cleaned_description']].head()

,description,cleaned_description
0,"<div data-testid=""careerPage""><p>At Vidyard, w...","At Vidyard, we make life easier for sellers, m..."
1,"<div data-testid=""careerPage""><p><span><em><st...",Version française disponible ci-bas. We’re loo...
2,"<div data-testid=""careerPage""><p><strong>Who w...",Who we are: We are a bunch of people who reall...
3,"<div data-testid=""careerPage""><section><div>Ax...",Axonify is the frontline enablement solution t...
4,"<div data-testid=""careerPage""><section><div><b...",About Us: Deep Genomics is a startup company...


#### making them all lower case for NLM modelling, can remove it if no need

In [35]:
def further_clean_text(text):
    # Convert to lowercase
    text = text.lower()

    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Remove numbers (if needed)
    text = re.sub(r'\d+', '', text)

    # Remove extra whitespace
    text = ' '.join(text.split())

    return text

# Apply the further cleaning function to the cleaned_description column
df['cleaned_description'] = df['cleaned_description'].apply(further_clean_text)

# Display the first few rows of the further cleaned descriptions
df[['description', 'cleaned_description']].head()


,description,cleaned_description
0,"<div data-testid=""careerPage""><p>At Vidyard, w...",at vidyard we make life easier for sellers mar...
1,"<div data-testid=""careerPage""><p><span><em><st...",version française disponible cibas we’re looki...
2,"<div data-testid=""careerPage""><p><strong>Who w...",who we are we are a bunch of people who really...
3,"<div data-testid=""careerPage""><section><div>Ax...",axonify is the frontline enablement solution t...
4,"<div data-testid=""careerPage""><section><div><b...",about us deep genomics is a startup company th...


In [36]:
df.head()

,organization_name,title,locations,organization_topics,organization_industry_tags,url,has_description,description,City,State/Province,Country,Is_Remote,consolidatedJobTitle,cleaned_description
0,Vidyard,"Director of Sales, Enterprise Acquisition","United States', 'Canada', 'Remote","['B2B', 'Media & entertainment', 'Productivity...","[Media, Software]",https://www1.communitech.ca/companies/vidyard/...,True,"<div data-testid=""careerPage""><p>At Vidyard, w...",United States','Canada',',True,Director,at vidyard we make life easier for sellers mar...
1,Hootsuite,Senior Software Developer | Développeur logici...,"Toronto, ON, Canada', 'Vancouver, BC, Canada","['500+', 'Advertising/marketing', 'B2B', 'Soft...","[Information Technology, Media, Sales and Mark...",https://www1.communitech.ca/companies/hootsuit...,True,"<div data-testid=""careerPage""><p><span><em><st...","Toronto, ON, Canada', 'Vancouver",BC,Canada,False,Senior,version française disponible cibas we’re looki...
2,Semios,Chemistry Co-op,"Vancouver, BC, Canada","['Agriculture', 'B2B', 'C100 alumni', 'Data & ...",[Agriculture and Farming],https://www1.communitech.ca/companies/semios/j...,True,"<div data-testid=""careerPage""><p><strong>Who w...",Vancouver,BC,Canada,False,Co-op,who we are we are a bunch of people who really...
3,Axonify,Senior Talent Acquisition Partner (15 months c...,"Toronto, ON, Canada', 'Waterloo, ON, Canada","['B2B', 'Data & analytics', 'Education', 'Fema...","[Education, Information Technology, Software]",https://www1.communitech.ca/companies/axonify/...,True,"<div data-testid=""careerPage""><section><div>Ax...","Toronto, ON, Canada', 'Waterloo",ON,Canada,False,Senior,axonify is the frontline enablement solution t...
4,Deep Genomics,(Senior) Research Scientist - Statistical Gene...,"Toronto, ON, Canada","['50-99', 'AI/ML', 'B2B', 'Biotechnology', 'Me...","[Biotechnology, DeepTech, Health, Software]",https://www1.communitech.ca/companies/deep-gen...,True,"<div data-testid=""careerPage""><section><div><b...",Toronto,ON,Canada,False,Senior,about us deep genomics is a startup company th...
